# Lecture 7: Working with Claude API and Reasoning Models

### Overview

Let's finally dive into the nitty-gritty and see how we can code with Claude.

This notebook explores the Anthropic Claude API, focusing on how to leverage both standard chat capabilities and advanced reasoning features. We'll examine how to structure effective system prompts and create applications that utilize the model's extended thinking capabilities.

### Objectives
- Set up and configure the Anthropic Claude API
- Create tailored system prompts for specific use cases
- Compare standard responses with reasoning-enhanced outputs
- Analyze the "thinking" process in reasoning models

### Challenge
Take the code we have used in this lecture and see if you can beat my overall score. Let's turn this into a friendly competition for all those tackling this project and compare the results you get :D

Place your submission in the community-contributions folder by creating a folder with your name in it.

### Version History

Below you will find any updates I have made to the notebooks.

**Please note** that some of the code may be different to what you have seen in the lectures as there may be bug fixes or improvements but the general objective of the notebook will remain the same.

| Date | Version | Description of Changes |
|------|---------|------------------------|
| 2025-03-01 | v1.0 | Initial version |



#### Step 1: Import libraries and load the environment variables

In [1]:
import os
import anthropic
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if ANTHROPIC_API_KEY is None:
    raise Exception("API key is missing")

#### Step 2: Call the API

In [2]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=1,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content[0].text)

The ocean's tears, ancient and deep,
From earth's minerals, secrets they keep.
Mountains dissolve, year after year,
Their essence carried to waters clear.
Salt of ages, in waves that roll,
The earth's long weeping, the sea's true soul.


#### Step 3: Customize the Prompt

Customer service representative

In [3]:
complaint = "You have over charged me for this product. I demand a refund."

In [4]:
system_message = "You are an expert customer service representative. Respond thoughtfully and informatively."

prompt = f"""
Write a brief professional email response to an unhappy customer with the following issue:

{complaint}

Requirements:
- Keep under 150 words
- Maintain a professional but empathetic tone
- Acknowledge the issue clearly
- Offer a specific solution
- Include a follow-up action
"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
)

chat_response = message.content[0].text
display(Markdown(chat_response))

# Subject: Regarding Your Refund Request - We're Here to Help

Dear Valued Customer,

Thank you for bringing this overcharging issue to our attention. I sincerely apologize for this error and understand your frustration.

I've reviewed your account and can confirm the discrepancy. We take billing accuracy very seriously, and I'm processing a full refund for the overcharged amount immediately. You should see this reflected in your account within 3-5 business days.

As a token of our apology, I'm also adding a 10% discount code for your next purchase: THANKYOU10.

To prevent similar issues in the future, our billing team is conducting a thorough review of our pricing system.

Please let me know if you haven't received your refund by [date]. I'm available at [contact information] if you have any questions.

Sincerely,
[Your Name]
Customer Service Representative

#### Step 4: Rate the response using a reasoning model

In [5]:
system_message = "You are an expert customer service representative. Evaluate the email response to be sent to a customer complaint."

reasoning_prompt = f"""
A customer has made a complaint.

{complaint}

Evaluate this customer service email response.

{chat_response}

Rate each category with ONE LINE explanation:

1. Professionalism (1-10): [Score] - [One-line explanation]
2. Empathy (1-10): [Score] - [One-line explanation]
3. Problem Resolution (1-10): [Score] - [One-line explanation]
4. Clarity (1-10): [Score] - [One-line explanation]
5. Brand Representation (1-10): [Score] - [One-line explanation]

TOTAL SCORE: [X/100]
"""

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=5000,
    thinking={
        "type": "enabled",
        "budget_tokens": 3000
    },
    temperature=1,
    system=system_message,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": reasoning_prompt
                }
            ]
        }
    ]
)
reasoning_response = message.content
print(reasoning_response)

[ThinkingBlock(signature='ErUBCkYIBBgCIkCYBpzeu+A5ejgFBek+V+ECp/8vs2nG7kOVnj+S2cTnmxBgQj3Thva2jsJ93MmQprsrU590VDa46IgRfX9Ihv7+Egxg0epnKv199laqkzkaDDfxzLxKDLr35Gl0syIweBfquNwFx1fbuunUAZi8bj5f1eHwn5Q4ubRZq4/CJpccX3X6Ia6fQplK5dEHmh4xKh1FKgHFOT/wBDM+l2f8bNIH2N3yYNU7mfMxiVEsWBgC', thinking='Let me evaluate this customer service email response across the five categories:\n\n1. Professionalism (1-10):\nThe email maintains a professional tone throughout, using appropriate language and formatting. It has a clear subject line, greeting, and sign-off. The representative identifies themselves as a customer service representative and provides contact information. The language is respectful and business-appropriate. There are no spelling or grammatical errors. The structure is organized and follows standard business email format.\nScore: 9/10 - The email is highly professional with appropriate structure, tone, and language.\n\n2. Empathy (1-10):\nThe response starts with an acknowledgment of the iss

In [6]:
display(Markdown(reasoning_response[1].text))

# Evaluation of Customer Service Email Response

1. Professionalism (1-10): 9 - Maintains formal business tone with appropriate structure, clear subject line, and proper closing.

2. Empathy (1-10): 9 - Immediately acknowledges the issue, offers a sincere apology, and validates the customer's frustration.

3. Problem Resolution (1-10): 10 - Provides clear confirmation of the error, immediate refund action, timeline, compensation, and preventative measures.

4. Clarity (1-10): 8 - Clear communication about resolution steps, though the placeholder fields ([date] and [contact information]) need to be completed.

5. Brand Representation (1-10): 9 - Demonstrates company values through accountability, generosity (10% discount), and commitment to improvement.

TOTAL SCORE: 90/100

This is an excellent customer service response that addresses the complaint directly, takes responsibility, offers immediate resolution with a clear timeline, and provides additional compensation. The response turns a negative situation into a potentially positive customer experience, though it would be even better with the placeholder information completed.